In [ ]:
!pip install ultralytics pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /content/drive/MyDrive/Alerts

/content/drive/MyDrive/Alerts


In [ ]:
from collections import defaultdict
import io
from PIL import Image
from ultralytics import YOLO
import email, smtplib, ssl
import pymongo
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime
import os

# for training
# model = YOLO('yolov8m.yaml')
# results = model.train(data="config.yaml",epochs=10)
# print(results)

def email(item:str):
    path = str(os.getcwd())+"/runs/detect"
    array = []
    cur_dict = {}

    def convertToNum(input:str):

        digits = len(input)
        fnum = 0
        length = digits
        for i in range(digits):
            fnum += (ord(input[i])-48)*10**(length-1)
            length-=1
        return fnum


    for entry in os.scandir(path):
        client = pymongo.MongoClient("MongoDB Credentials")
        db = client.get_database('test_data')
        collection_name = db.test_record

        try:
            if(str(entry).index('predict')):
                text =  str(entry)[str(entry).index('predict'):len(str(entry))-2]
                num = ''
                i=0
                while(i!=len(text)):
                    if(text[i]=='t' and i+1!=len(text)):
                        i+=1
                        while(i!=len(text)):
                            num+=text[i]
                            i+=1
                    else:
                        i+=1
                array+=[convertToNum(num)]

        except ValueError:
            pass


    latest_dir = path+"/predict/"
    filename= ''
    if(array[0]!=0):

        array.sort()
        latest_dir = path+'/predict'+str(array[-1])+"/"
        filename = os.listdir(latest_dir)[0]
        latest_dir+=filename
    else:
        latest_dir = path+'/predict'+"/"
        filename = os.listdir(latest_dir)[0]
        latest_dir+=filename

    subject = "Emergency alert !!"
    body = f"{item} detected !!"
    sender_email = "user_email"
    receiver_email = "receiver_email"
    password = 'two-fath password'

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject



    message.attach(MIMEText(body, "plain"))

    with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    encoders.encode_base64(part)


    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    message.attach(part)
    text = message.as_string()

    context = ssl.create_default_context()
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.ehlo()
        server.starttls(context=context)
        server.ehlo()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    im = Image.open(latest_dir)
    image_bytes = io.BytesIO()
    im.save(image_bytes, format='PNG')

    cur_dict['TimeStamp'] = datetime.now()
    cur_dict['Screenshot'] = image_bytes.getvalue()
    cur_dict['Alert_Type'] = 'High'

    collection_name.insert_one(cur_dict)

path = os.getcwd()
model = YOLO(path+"/weights/best.pt")
res:list
res = model.predict(source=path+'/malorian-arms-3516-weapon-cyberpunk-2077-wiki-guide.png',save=True,conf=0.5)
items = ''
if(res[0].names):
    for i in res[0].names:
        items += res[0].names[i]
    email(items)


image 1/1 /content/drive/MyDrive/Alerts/malorian-arms-3516-weapon-cyberpunk-2077-wiki-guide.png: 384x640 1 handgun, 60.9ms
Speed: 0.6ms preprocess, 60.9ms inference, 36.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict15


In [ ]:
%matplotlib inline
import pymongo
import matplotlib.pyplot as plt
from PIL import Image
import io

client = pymongo.MongoClient("MongoDB Credentials")
db = client.get_database('Alerts')
collection_name = db.High_Alert
image = collection_name.find_one()

pil_img = Image.open(io.BytesIO(image['Screenshot']))
plt.imshow(pil_img)
plt.show()